<a href="https://colab.research.google.com/github/imhyunho99/2023-1--Advanced_python/blob/main/pylib_044~053.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#07 Data control
*7-1 pickle module*



https://docs.python.org/ko/3/library/pickle.html

In [1]:
import pickle

data = {}
data[1] = {'no': 1, 'subject': '안녕 피클', 'content': '피클은 매우 간단합니다.'}

with open('data.p', 'wb') as f: #Binary writing
  pickle.dump(data, f)

In [2]:
!cat data.p

안녕 피클��content��!피클은 매우 간단합니다.�us.

In [6]:
with open("data.p", 'rb') as f:
  load_data = pickle.load(f)


In [5]:
import pickle


def get_all_data():
    try:
        with open("data.p", 'rb') as f: 
            return pickle.load(f)
    except FileNotFoundError:
        return {}


def add_data(no, subject, content):
    data = get_all_data()
    assert no not in data
    data[no] = {'no': no, 'subject': subject, 'content': content}
    with open('data.p', 'wb') as f:
        pickle.dump(data, f)


def get_data(no):
    data = get_all_data()
    return data[no]


# 데이터저장
#add_data(1, '안녕 피클', '피클은 매우 간단합니다.') AssertionError: 

# 데이터조회
data = get_data(1)
print(data['no'])
print(data['subject'])
print(data['content'])


add_data(2, '잘자 피클', '피클은 멋있어.')


1
안녕 피클
피클은 매우 간단합니다.


In [6]:
get_all_data()

{1: {'no': 1, 'subject': '안녕 피클', 'content': '피클은 매우 간단합니다.'},
 2: {'no': 2, 'subject': '잘자 피클', 'content': '피클은 멋있어.'}}

*7-2 copyreg module*


*   copyreg.pickle()




In [11]:
import pickle

class Student:
    def __init__(self, name, age):
        self.name = name
        self.age = age


a = Student('임철희', 27)

with open('student.p', 'wb') as f:
    pickle.dump(a, f)

In [16]:
#만약 새로운 속성이 추가되었다면? ==> 
#student.dummy
#write code

import pickle
import copyreg


class Student:
    def __init__(self, name, age):
        self.name = name
        self.age = age


def pickle_student(student):
    kwargs = student.__dict__
    return unpickle_student, (kwargs, )


def unpickle_student(kwargs):
    return Student(**kwargs)


copyreg.pickle(Student, pickle_student)

a = Student('임철희', 27)
with open('student.p', 'wb') as f:
    pickle.dump(a, f)

In [18]:
class Student:
    def __init__(self, name, age):
        self.name = name
        self.age = age
        self.dummy = 'dummy'  # dummy 속성을 새로 추가!


def pickle_student(student):
    kwargs = student.__dict__
    return unpickle_student, (kwargs, )


def unpickle_student(kwargs):
    return Student(**kwargs)


copyreg.pickle(Student, pickle_student)

with open('student.p', 'rb') as f:
    student = pickle.load(f)  # unpickle_student() 함수를 호출한다.

student.__dict__  # 오류가 발생하지 않고 'dummy' 가 출력된다.

{'name': '임철희', 'age': 27, 'dummy': 'dummy'}

*7-3
 shelve module*


*   copyreg.pickle()




In [20]:
import shelve


def save(key, value):
    with shelve.open('shelve.dat') as d:
        d[key] = value


def get(key):
    with shelve.open('shelve.dat') as d:
        return d[key]


save('number', [1, 2, 3, 4, 5])
print(get('number'))  # [1, 2, 3, ,4, 5] 출력|
save('number', [1, 2, 3])
get('number')

[1, 2, 3, 4, 5]


[1, 2, 3]

*7-4 sqlite3 module*


*    sqlite3.connect()
*    c.execute()
*    c.fetchall(),c.fetchone()
*    conn.commit()
*    conn.rollback()
*    conn.close()




https://sqlitebrowser.org/dl

In [ ]:
import sqlite3
conn = sqlite3.connect('blog.db')

c = conn.cursor()
c.execute('''CREATE TABLE blog (id integer PRIMARY KEY, subject text, content text, date text)''')

c.execute("INSERT INTO blog VALUES (1, '첫 번째 블로그', '첫 번째 포스팅입니다.', '201914074')")
c.execute("INSERT INTO blog VALUES (2, '두 번째 블로그', '두 번째 포스팅입니다.', '201914074')")

_id = 3
subject = "세 번째 블로그"
content = "세 번째 블로그입니다."
date = "20190827"
c.execute("INSERT INTO blog VALUES (%d, '%s', '%s', %s)" % (_id, subject, content, date))

_id = 4
subject = "네 번째 블로그"
content = "네 번째 블로그입니다."
date = "20190827"
c.execute("INSERT INTO blog VALUES (?, ?, ?, ?)",  (_id, subject, content, date))

c.execute("INSERT INTO blog VALUES (:id, :subject, :content, :date)", {"id": 5, "subject": "다섯 번째 블로그", "content": "다섯 번째 블로그입니다.", "date": "20190827"})

In [89]:
c.execute('SELECT * FROM blog')
all = c.fetchall()
all

[(1, '최초의 블로그', '첫 번째 포스팅입니다.', '201914074'),
 (2, '두 번째 블로그', '두 번째 포스팅입니다.', '201914074'),
 (3, '세 번째 블로그', '세 번째 블로그입니다.', '20190827'),
 (4, '네 번째 블로그', '네 번째 블로그입니다.', '20190827')]

In [91]:
all = c.fetchall()
all #이미 쿼리를 한번 날려서 더이상 불러올게 없음

[]

In [92]:
c.execute("UPDATE blog SET subject='최초의 블로그' WHERE id=1")


In [93]:
c.execute('SELECT * FROM blog WHERE id=1')
one = c.fetchone()
print(one)

(1, '최초의 블로그', '첫 번째 포스팅입니다.', '201914074')


In [94]:
c.execute('DELETE FROM blog WHERE id=5')#계속 커서 위치 반환해주는중

In [95]:
conn.commit()
#conn.rollback() 커밋 전상태로 롤백

In [96]:
conn.close()

In [99]:
conn = sqlite3.connect('blog.db')
c = conn.cursor()
c.execute("INSERT INTO blog (subject, content, date) VALUES (?, ?, ?)",("자동증가", "자동증가 블로그", "20191030"))
all = c.fetchall()

In [100]:
import sqlite3
import time
#데코레이터를 사용하면 확실히 짧아진다.
def with_cursor(original_func):
    def wrapper(*args, **kwargs):
        conn = sqlite3.connect('blog.db')
        conn.row_factory = sqlite3.Row
        c = conn.cursor()
        rv = original_func(c, *args, **kwargs)
        conn.commit()
        conn.close()            
        return rv
    return wrapper


@with_cursor
def get_blog_list(c):
    c.execute("SELECT * FROM blog")
    return c.fetchall()


@with_cursor
def add_blog(c, subject, content):
    c.execute("INSERT INTO blog (subject, content, date) VALUES (?, ?, ?)", 
        (subject, content, time.strftime('%Y%m%d')))


@with_cursor
def read_blog(c, _id):
    c.execute("SELECT * FROM blog WHERE id=?", (_id,))
    return c.fetchone()


@with_cursor
def modify_blog(c, _id, subject, content):
    c.execute("UPDATE blog SET subject=?, content=? WHERE id=?", 
        (subject, content, _id))


@with_cursor
def remove_blog(c, _id):
    c.execute("DELETE FROM blog WHERE id=?", (_id,))

In [101]:
for blog in get_blog_list():
    print(blog)
    print(blog["subject"])

최초의 블로그
두 번째 블로그
세 번째 블로그
네 번째 블로그


In [102]:
read_blog(4)

In [87]:
get_blog_list()

#07 Data Zip and Store 


*   zlib module
*   gzip module
*   bz2 module
*   lzma module
*   zipfile module
*   tarfile module




In [103]:
import zlib
#인코딩 방식을 정해줘야함

data = "Life is too short, You need python." * 10000 #대략 10000*35개개
compress_data = zlib.compress(data.encode(encoding='utf-8')) #압축하고 인코딩
print(len(compress_data))  # 1077 출력

org_data = zlib.decompress(compress_data).decode('utf-8') #압축풀고 디코딩
print(len(org_data))  # 350000 출력

1077
350000


In [107]:
import gzip #파일 압축

data = "Life is too short, you need python." * 10000

with gzip.open('data.txt.gz', 'wb') as f:
    f.write(data.encode('utf-8'))  # 저장한 파일의 크기는 1097바이트

with gzip.open('data.txt.gz', 'rb') as f:
    read_data = f.read().decode('utf-8')

assert data == read_data

In [108]:
import bz2
#스레드 환경에서 안정

data = "Life is too short, You need python." * 10000
compress_data = bz2.compress(data.encode(encoding='utf-8'))
print(len(compress_data))  # 163 출력

org_data = bz2.decompress(compress_data).decode('utf-8')
print(len(org_data))  # 350000 출력

assert data == org_data

163
350000


In [109]:
import lzma
#스레드 환경 불안정

data = "Life is too short, You need python." * 10000
compress_data = lzma.compress(data.encode(encoding='utf-8'))
print(len(compress_data))  # 220 출력

org_data = lzma.decompress(compress_data).decode('utf-8')
print(len(org_data))  # 350000 출력

assert data == org_data

220
350000


In [121]:
!ehco > a.txt
!ehco > b.txt
!ehco > c.txt

/bin/bash: ehco: command not found
/bin/bash: ehco: command not found
/bin/bash: ehco: command not found


In [122]:
import zipfile

# 파일 합치기
with zipfile.ZipFile('mytext.zip', 'w') as myzip:
    myzip.write('a.txt')
    myzip.write('b.txt')
    myzip.write('c.txt')

# 해제하기
with zipfile.ZipFile('mytext.zip') as myzip:
    myzip.extractall()


In [123]:
import tarfile
#확장자를 추가하면 그 알고리즘으로 압축해줌줌
with tarfile.open('mytext.tar', 'w') as mytar:
    mytar.add('a.txt')
    mytar.add('b.txt')
    mytar.add('c.txt')

# 여러파일 해제하기
with tarfile.open('mytext.tar') as mytar:
    mytar.extractall()